**Загрузим все части нашего датафрейма**

In [8]:
import pandas as pd
heroes = pd.read_csv('../input/heroes/heroes', encoding='ISO-8859-1', error_bad_lines=False)
lobby_game_mode = pd.read_csv('../input/lobby-game-mode/lobby_game_mode', encoding='ISO-8859-1', error_bad_lines=False)
Times_of_Day = pd.read_csv('../input/times-of-day/Times_of_Day', encoding='ISO-8859-1', error_bad_lines=False)

In [9]:
first_df = pd.read_csv('../input/firstf/first_df', encoding='ISO-8859-1', error_bad_lines=False)

In [10]:
y = pd.read_csv('../input/yyyyyyyyyyyyyyyyyyyyy/Y', encoding='ISO-8859-1', error_bad_lines=False, header=None)

Удалим лишние стобцы

In [11]:
y.drop([0], axis=1, inplace=True)
y

,1
0,1240
1,1355
2,1168
3,1425
4,1311
...,...
2980245,2281
2980246,2875
2980247,2598
2980248,2297


In [12]:
heroes.drop(['Unnamed: 0'], axis=1, inplace=True)
lobby_game_mode.drop(['Unnamed: 0'], axis=1, inplace=True)
Times_of_Day.drop(['Unnamed: 0'], axis=1, inplace=True)
first_df.drop(['Unnamed: 0'], axis=1, inplace=True)

**Создадим из каждой части(кроме целевого признака) разреженную матрицу**

In [13]:
from scipy.sparse import csr_matrix, hstack

In [14]:
sparse_heroes = csr_matrix(heroes)
sparse_lobby_game_mode = csr_matrix(lobby_game_mode)
sparse_Times_of_Day = csr_matrix(Times_of_Day)

In [15]:
first_df.drop(['lobby_type', 'game_mode'], axis=1, inplace=True)
first_df

,start_time,avg_mmr,day_in_week
0,1598044669,2632.000000,5
1,1598044529,1805.000000,5
2,1598044494,4701.473684,5
3,1598044469,4089.000000,5
4,1598044460,1549.000000,5
...,...,...,...
2980245,1583268409,3115.000000,2
2980246,1583268416,2154.842105,2
2980247,1583268421,3144.000000,2
2980248,1583268415,1763.052632,2


In [16]:
full_df = hstack([first_df, sparse_heroes, sparse_lobby_game_mode, sparse_Times_of_Day]).tocsr()

**Посчитаем среднюю абсолютную ошибку наивного предсказания(будем всегда предсказывать медиану)**

In [17]:
y.median()

1    2267.0
dtype: float64

In [18]:
y_median=2267.0

In [19]:
%%time
X_train, X_test, y_train, y_test = train_test_split(full_df, y, test_size=0.8, random_state=0)
pred_one_value = []
for i in range(2384200):
    pred_one_value.append([y_median])

CPU times: user 2.75 s, sys: 202 ms, total: 2.95 s
Wall time: 2.96 s


In [20]:
print(mean_absolute_error(y_test, pred_one_value))

443.03043452730475


**Проделаем тоже самое со средним**

In [21]:
y.mean()

1    2315.147882
dtype: float64

In [22]:
y_mean = 2315.147882

In [23]:
%%time
X_train, X_test, y_train, y_test = train_test_split(full_df, y, test_size=0.8, random_state=0)
pred_one_value = []
for i in range(2384200):
    pred_one_value.append([y_mean])

CPU times: user 2.84 s, sys: 203 ms, total: 3.05 s
Wall time: 3.05 s


In [24]:
print(mean_absolute_error(y_test, pred_one_value))

444.64414639589313


**Результат чуть лучше у медианы**

**Импортируем нужные для обучения библиотеки и напишем функции для обучения различных алгоритмов**

In [6]:
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split

In [7]:
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error
import numpy as np

In [31]:
def learning_XGB(n_est=1000, lr=0.05, md=7, ts=0.8, rs=0):
    print(n_est, lr, md, ts, end=' ')
    reg_xgb = XGBRegressor(n_estimators=n_est, learning_rate=lr, gamma=0, subsample=0.75,max_depth=md,random_state=5,
                   min_child_weight=1,colsample_bytree=0.8)
    X_train, X_test, y_train, y_test = train_test_split(full_df, y, test_size=ts, random_state=rs)
    reg_xgb.fit(X_train, y_train)
    predictions = reg_xgb.predict(X_test)
    metric = mean_absolute_error(y_test, predictions)
    print(metric)
    results_XGB.append((metric, n_est, lr, md, ts))

In [23]:
from sklearn.ensemble import RandomForestRegressor
def learning_random_forest(n_est=100, md=7):
    print(n_est, md, end=' ')
    regr = RandomForestRegressor(max_depth=md, n_estimators=n_est)
    X_train, X_test, y_train, y_test = train_test_split(full_df, y, test_size=0.8, random_state=0)
    regr.fit(X_train, y_train)
    predictions = regr.predict(X_test)
    metric = mean_absolute_error(y_test, predictions)
    print(metric)
    results_rf.append((metric, n_est, md))

In [24]:
from sklearn.ensemble import GradientBoostingRegressor
def learning_GradientBoosting(n_est=100, md=7, lr=0.1):
    print(n_est, lr, md, end=' ')
    regr = GradientBoostingRegressor(max_depth=md, n_estimators=n_est)
    X_train, X_test, y_train, y_test = train_test_split(full_df, y, test_size=0.97, random_state=0)
    regr.fit(X_train, y_train)
    predictions = regr.predict(X_test)
    metric = mean_absolute_error(y_test, predictions)
    print(metric)
    results_grad_boosting.append((metric, n_est, lr, md))

In [4]:
def learning_linear(ts=0.8):
    print(ts, end=' ')
    from sklearn.linear_model import LinearRegression
    reg = LinearRegression()
    X_train, X_test, y_train, y_test = train_test_split(full_df, y, test_size=ts, random_state=0)
    reg.fit(X_train, y_train)
    predictions = reg.predict(X_test)
    print(len(predictions))
    metric = mean_absolute_error(y_test, predictions)
    results_linear.append((metric, ts))
    print(metric)

**Создадим списки для сохранения результатов работы алгоритмов**

In [26]:
results_XGB = [432,
 433.0227,
 (434.86605677549466, 500),
 (430.07820174550193, 1500),
 (428.5033875546514, 2000),
 (422.61867876379694, 5000),
 (430.7697705048515, 800, 0.1),
 (427.7687350481612, 1500, 0.1),
 (430.7657547477254, 800, 0.2),
 (448.61037582660856, 800, 0.5),
 (434.4145672227297, 800, 0.3),
 (440.9305807259365, 800, 0.0078125, 7),
 (437.67679178637707, 800, 0.015625, 7),
 (434.6860178017969, 800, 0.03125, 7),
 (432.18326556042604, 800, 0.0625, 7),
 (430.4772854392218, 800, 0.125, 7),
 (442.8380501377253, 800, 0.125, 1),
 (439.55859512705644, 800, 0.125, 2),
 (438.18399326743975, 800, 0.125, 3),
 (436.84285627943297, 800, 0.125, 4),
 (435.19394660516997, 800, 0.125, 5),
 (432.95661956759085, 800, 0.125, 6),
 (430.4772854392218, 800, 0.125, 7),
 (427.5080559266423, 800, 0.125, 8),
 (424.4342987356769, 800, 0.125, 9),
 (421.15249778662496, 800, 0.125, 10),
 (417.53611720694215, 800, 0.125, 11),
 (413.9346006081626, 800, 0.125, 12),
(800, 0.125, 8, 402.8108188798805),
(800, 0.125,13, 386.05734157794893),
(800, 0.125,18, 373.5684148064328),
(800, 0.125, 23, 370.99645250621074)]
results_linear = []
results_rf =[]

In [27]:
results_grad_boosting = []

**Обучим случайный лес с разным числом деревьев**

In [28]:
%%time
for i in range(100, 400, 100):
    learning_random_forest(n_est=i)

100 7 

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:6: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


427.7648822085617
200 7 

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:6: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


427.79215085718477
300 7 

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:6: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


427.78713197275135
CPU times: user 1h 33min, sys: 1.45 s, total: 1h 33min 2s
Wall time: 1h 34min 27s


**Особой разницы в качестве не наблюдается, лучше всего себя показал лес из 100 деревьев с результатом MAE = 427.76 сек.**

Обучим градиентный бустинг с разной максимальной глубиной деревьев

In [29]:
%%time
for i in range(5, 16, 5):
    learning_GradientBoosting(md=i)

100 0.1 5 

/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


420.1015799105426
100 0.1 10 

/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


418.42158952591615
100 0.1 15 

/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


418.9376060249802
CPU times: user 13min 32s, sys: 1.78 s, total: 13min 34s
Wall time: 13min 49s


**Лучшее качество наблюдается при макс. глубине 10**

**Попробуем также подобрать скорость обучения**

In [32]:
%%time
for i in np.arange(-2, 2):
    learning_GradientBoosting(lr=pow(10.0,i), md=10)

100 0.01 10 

/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


418.44822013660223
100 0.1 10 

/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


418.4504681293619
100 1.0 10 

/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


418.4612863691927
100 10.0 10 

/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


418.5004424825694
CPU times: user 14min 26s, sys: 1.69 s, total: 14min 27s
Wall time: 14min 39s


**Особой разницы не наблюдается, однако лучший результат был показан при наименьшем значении lr. Посмотрим на качество при еще более низком значении lr.**

In [33]:
%%time
for i in np.arange(-5, -2):
    learning_GradientBoosting(lr=pow(10.0,i), md=10)

100 1e-05 10 

/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


418.4757660736668
100 0.0001 10 

/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


418.44451784155973
100 0.001 10 

/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


418.46081417502467
CPU times: user 11min 3s, sys: 1.61 s, total: 11min 5s
Wall time: 11min 17s


**Лучшее качество наблюдается при lr=0.0001. При нем достигается значение MAE=418.44 сек.**

**Попробуем линейную регрессию**

In [28]:
%%time
learning_linear()

0.8 2384200
441.60563400572266
CPU times: user 1.48 s, sys: 121 ms, total: 1.6 s
Wall time: 1.3 s


**Худший результат. Лишь на несколько секунд превосходит качество наивного прогноза.**

**Наконец обучим XGBoost**

In [32]:
%%time
learning_XGB(n_est=800, lr=0.125, md=23)

800 0.125 23 0.8 370.99645250621074
CPU times: user 2h 37min 31s, sys: 28.6 s, total: 2h 37min 59s
Wall time: 41min 10s


**В процессе подбора параметров, который в этот ноутбук не вошел, выяснилось, что из всех опробованных параметров лучшими оказались: n_estimators=800, learning_rate=0.125, max_depth=23, остальные параметры по умолчанию**

Лучший результат показал XGBoost(MAE=371 сек.). Таким образом нам удалось улучшить качество прогноза по сравнению с наивным на 72 секунды